# Combine multiple outputs for visualisation

In [ ]:
import os 
import numpy as np
from distutils.dir_util import copy_tree

Parameters for combining models for Paraview visualisation

In [ ]:
# Specify most recent time in Ma BP
startMa = 16
# Specify deepest time in Ma BP
endMa = 20
# Output steps
steps = 50
# XDMF file
xdmf_file = 'model.xdmf'

model_folder = 'model_out'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

Getting the `XMF` files

In [ ]:
timeframe = np.flipud(np.linspace(startMa,endMa,endMa-startMa+1,dtype=int))

for k in range(len(timeframe)):
    time = timeframe[k]

    xmf_folder = os.path.join(model_folder,'xmf_'+str(time))
    if not os.path.exists(xmf_folder):
        os.makedirs(xmf_folder)

    fromDirectory = 'model'+str(time)+'Ma/xmf'
    toDirectory = os.path.join(model_folder,'xmf_'+str(time))
    files = copy_tree(fromDirectory, toDirectory)

    for f in files:
        reading_file = open(f, "r")

        new_file_content = ""
        for line in reading_file:
            stripped_line = line.strip()
            new_line = stripped_line.replace('>h5', '>model'+str(time)+'Ma/h5')
            new_file_content += new_line +"\n"

        reading_file.close()

        writing_file = open(f, "w")
        writing_file.write(new_file_content)
        writing_file.close()
    
    
    nbfile = len(files)

Building the `XDMF` file that will be launch in Paraview

In [ ]:
f = open(xdmf_file, 'w')

f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
f.write('<!DOCTYPE Xdmf SYSTEM "Xdmf.dtd">\n')
f.write('<Xdmf Version="2.0" xmlns:xi="http://www.w3.org/2001/XInclude">\n')
f.write(' <Domain>\n')
f.write('    <Grid GridType="Collection" CollectionType="Temporal">\n')

for s in range(len(timeframe)):
    nb = steps
    if s == 0:
        nb = 0
    for p in range(nb,nbfile,steps):
        xmf_file = model_folder+'/xmf_'+str(timeframe[s])+'/gospl' + str(p) + '.xmf'
        f.write(
            '      <xi:include href="%s" xpointer="xpointer(//Xdmf/Domain/Grid)"/>\n'
            % xmf_file
        )

f.write('    </Grid>\n')
f.write(' </Domain>\n')
f.write('</Xdmf>\n')
f.close()